# ビットコインの価格予測プログラム

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU
import yfinance as yf

def load_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

def add_features(data, window_size):
    data['SMA'] = data['Close'].rolling(window=window_size).mean()
    data.dropna(inplace=True)
    return data

def preprocess_data(data, window_size):
    data = data[['Close', 'SMA']].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data)
    x, y = [], []
    for i in range(window_size, len(data)):
        x.append(data[i - window_size:i])
        y.append(data[i, 0])
    x, y = np.array(x), np.array(y)
    x = np.reshape(x, (x.shape[0], x.shape[1], x.shape[2]))
    return x, y, scaler

def create_model(input_shape):
    model = Sequential()
    model.add(GRU(50, return_sequences=True, input_shape=input_shape))
    model.add(GRU(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def predict_price(model, data, scaler, window_size):
    scaled_data = scaler.transform(data[-window_size:])
    x_test = np.array(scaled_data)
    x_test = np.reshape(x_test, (1, x_test.shape[0], x_test.shape[1]))
    prediction = model.predict(x_test)
    prediction = scaler.inverse_transform(np.hstack((prediction, np.zeros((prediction.shape[0], 1)))))[0][0]
    return prediction

# Parameters
ticker = "BTC-USD"
start_date = "2016-01-01"
end_date = "2021-09-01"
window_size = 60

# Load and preprocess data
data = load_data(ticker, start_date, end_date)
data = add_features(data, window_size)
x, y, scaler = preprocess_data(data, window_size)

# Time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)
model = create_model((x.shape[1], x.shape[2]))
total_loss = 0

for train_index, test_index in tscv.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(x_train, y_train, batch_size=1, epochs=1)
    loss = model.evaluate(x_test, y_test)
    total_loss += loss

avg_loss = total_loss / 5
print(f"Average loss: {avg_loss}")

# Predict the next day's closing price


ImportError: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
                                  ^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/imp.py", line 243, in load_module
    return load_dynamic